## INF-616-1 - Atividade 1: máquina de vetor de suporte e validação cruzada

Professor: Ricardo Torres -- rtorres@ic.unicamp.br  
Monitor: Lucas David -- lucasolivdavid@gmail.com

Este *notebook* faz parte da disciplina INF-616 no curso de extensão MDC.  
Demais artefatos podem ser encontrados no moodle da disciplina: 
[moodle.lab.ic.unicamp.br/317](https://moodle.lab.ic.unicamp.br/moodle/course/view.php?id=317)

Instituto de Computação - Unicamp 2019

In [ ]:
from __future__ import print_function

from math import ceil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats

from sklearn import metrics
from sklearn.model_selection import train_test_split

import seaborn as sns

In [ ]:
np.random.seed(1082141)
sns.set()

## Lendo o conjunto de dados smart-debt-manager

In [ ]:
dataset = '/C:\Users\EMELFEL\OneDrive - Ericsson AB\Curso\Aprendiado de maquina supervisionado 2\ExercícioAtividade 1/smart-debt-manager.csv'
debt = pd.read_csv(dataset, index_col=0)

Amostra com os cinco primeiros clientes no conjunto:

In [ ]:
debt.head()

Descrição geral de características:

- OHXACT: id of a specific invoice
- CUSTOMER_ID: id of this invoice's customer
- OHENTDATE: date, in seconds from 1970, when the invoice was generated
- OHDUEDATE: date, in seconds from 1970, when the invoice is due
- OHINVAMT_DOC: total value of the invoice, in bangladeshi takas
- PAYMENT_DATE*: date, in seconds from 1970, when the invoice was actually paid
- PAYMENT_LABEL*: whether the invoice was paid before ('On time'), after ('Late') the due date or not at all ('')
- PAYMENT_LABEL2*: whether the invoice was paid before the due date ('On time'), up to 3 days after it ('Grace period'), after that ('Late') o not at all
- PAYMENT_COUNT*: number of payments for this invoice
- OHINVAMT_DOC_1*: total value value of the paid invoices, in ganbladeshi takas
- PAYMENT_AMOUNT*: amount paid by the customer regarding this invoice's period
- CSACTIVATED: date, in seconds from 1970, when the customer's account was activated
- COSTCENTER_ID: customer's costcenter
- TMCODE: customer's rateplan
- CSCLIMIT: customer's credit limit
- CASHRETOUR*: number of disputes won by the customer
- CHARGING_ENGINE_CODE: to which charging engine this invoice belongs to
- PAYMENT_METHOD_IND*: payment method used to pay such invoice, if any (bank transfer or direct payment)
- TIME_UNTIL_DUEDATE: time, in seconds, between invoice generation and due date
- TIME_AS_CUSTOMER: time, in seconds, between customer activation and due date
- IS_LATE*: whether the invoice was not paid in time
- GRACE_PERIOD*: whether the invoice was paid in the first 3 days after the due date
- GRACE_PERIOD2*: whether the invoice was paid in the first 10 days after the due date
- TOO_LATE*: whether the invoice was not paid up to 10 days after the due date
- MONTH: duedate's month
- INV_LAST_YEAR: total number of invoices for this customer in the 12 months prior to this invoice's due date
- INV_LAST_YEAR_LATE: total number of invoices for this customer in the 12 months prior to this invoice's due date that were not paid up to the due date
- INV_LAST_YEAR_CHARGE: total value of invoices for this customer in the 12 months prior to this invoice's due date
- INV_LAST_YEAR_PAID: total value paid by this customer in the 12 months prior to this invoice's due date
- INV_LATE_RATIO: proportion of late invoices over total invoices, for this customer, in the last 12 months
- INV_PAID_VALUE_RATIO: proportion of absolute total paid value over total charged value, for this customer, in the last 12 months

The features marked with an asterisk should only be used as target/validation values and not used for training, since such information wouldn't be available during training for a new invoice.

All data was collected from real entries covering a period of 10 years.
Most information comes from BSCS tables "ORDERHDR_ALL" and "CUSTOMER_ALL".
Tardiness features (IS_LATE, TOO_LATE etc) were calculated by checking whether a given customer had payments with enough value up to a threshold(0 for IS_LATE, 10 days for GRACE_PERIOD_2 etc) time after the due date of a given invoice.

The last 6 features were obtained by aggregating, over 12 months, all past invoices and payments related to a given customer in a giver moment (using, as reference, the due date of the observed invoice)

Dates were all transformed to seconds from 1970 in order to help numerical operations. All other data is displayed as it was collected from the database.

In [ ]:
identifiers = ['OHXACT', 'CUSTOMER_ID', 'COSTCENTER_ID', 'TMCODE', 'CHARGING_ENGINE_CODE']
post_payment_vars = ('PAYMENT_DATE PAYMENT_LABEL PAYMENT_LABEL2 PAYMENT_COUNT '
                     'OHINVAMT_DOC_1 PAYMENT_AMOUNT CASHRETOUR PAYMENT_METHOD_IND '
                     'IS_LATE GRACE_PERIOD GRACE_PERIOD2 TOO_LATE').split(' ')

### Preprocessamento

Vamos aplicar algumas operações sobre os dados para facilitar a manipulação adiante.

In [ ]:
# subtraímos os códigos 1 e 2 por 1, resultando em 0 e 1.
debt['CHARGING_ENGINE_CODE'] -= 1

# preenchemos todos os labels '' com 'not-paid' nas colunas de pagamento.
debt.fillna({ 'PAYMENT_LABEL': 'Not paid', 'PAYMENT_LABEL2': 'Not paid' }, inplace=True)

#### Distribuição das características no conjunto

Esta visualização permite relacionar cada característica do conjunto par-a-par,
onde a figura na posição $(i, j)$ contém a distribuição que a característica $i$ assume em relação à $j$.
A diagonal principal é a exceção, mostrando o histogram da variável $i$.

Primeiro, definimos todas as variáveis que irão aparecer no nosso gráfico de distribuição:

In [ ]:
inspecting_vars = ['OHINVAMT_DOC', 'CSCLIMIT', 'TIME_UNTIL_DUEDATE',
                   'TIME_AS_CUSTOMER', 'INV_PAID_VALUE_RATIO']

features = set(debt.columns) - set(identifiers) - set(post_payment_vars)
features.add('CHARGING_ENGINE_CODE')

target = 'PAYMENT_LABEL'

Agora selecionamos aleatoriamente um subconjunto, a fim de acelerar o processo:

In [ ]:
selected = np.arange(len(debt))
np.random.shuffle(selected)
selected = selected[:10000]

In [ ]:
sns.pairplot(debt.loc[selected].fillna(0),
             hue=target,
             diag_kind='hist',
             vars=inspecting_vars);

Destes gráficos, podemos observar algumas relações interessantes:

- A grande maioria das amostras apresenta um limite de crédito inferior ou igual à 1000
- As poucas amostras com alto `INV_PAID_VALUE_RATIO` vêm principalmente de novos e médio clientes
- Clientes novos parecem compor uma maior taxa de amostras `not-paid`

Um jeito mais certo enxuto é observar a **correlação absoluta** entre as variáveis do conjunto, que descrevem as relações lineares absolutas entre os pares de variáveis:

In [ ]:
correlations = debt.loc[selected, features].corr().abs()

plt.figure(figsize=(16, 12))
ax = sns.heatmap(correlations, linewidths=.5, cmap='YlGnBu',
                 annot=True, fmt='.2f',
                 xticklabels=features, yticklabels=features);

In [ ]:
plt.figure(figsize=(16, 4))
plt.title('Frequencia das classes em todo o conjunto (%i amostras)' % len(debt))
labels, counts = np.unique(debt.loc[selected, target], return_counts=True)
sns.barplot(labels, counts);

## Modelando o problema

Vamos definir o modelo que é alimentado por todas as variáveis pré-pagamento (ao qual temos acesso) que não são identificadores (específicos de um indivíduo ou característica categórica). A exceção é *CHARGING_ENGINE_CODE*, que se mantém por se tratar de um valor binário.

In [ ]:
print('Features utilizadas:', *features, sep='\n', end='\n\n')

### Restringindo o problema em duas classes
A informação alvo a ser predita é `PAYMENT_LABEL`. Como só vimos o SVC binário até agora, vamos binarizar o problema combinando as classes "Late" e "not-paid". Isso irá re-organizar nosso conjunto em duas classes: "on time" e "not on time".

In [ ]:
x = debt[features]
y = debt[target].copy()

y[y == 'Late'] = 'Not on time'
y[y == 'Not paid'] = 'Not on time'

In [ ]:
plt.figure(figsize=(16, 4))
plt.title('Frequencia das classes em todo o conjunto (%i amostras)' % len(debt))
labels, counts = np.unique(y, return_counts=True)
sns.barplot(labels, counts);

Separamos metade dos dados para testar nosso estimador:

In [ ]:
test_size = .5
train_samples = ceil((1 - test_size) * len(debt))
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=test_size,
                                                    random_state=8173)

A média e desvio padrão de cada característica é calculada. As amostras são então normalizadas a fim de remover translação e garantir um espalhamento similar.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

encoder = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler())

z_train = encoder.fit_transform(x_train)
z_test = encoder.transform(x_test)

## Visualizando os dados

Uma outra forma de visualizar conjuntos é rotacioná-los até que as direções que maximizam a variabilidade
dos dados (as componentes principais) estejam alinhadas com a base canônica $\{x, y, z\}$.  
Isso faz mais sentido sobre um espaço de características métricas.

In [ ]:
from sklearn.manifold import TSNE

encoder2D = TSNE(n_components=2)
w_train = encoder2D.fit_transform(z_train[:1000])
w_test = encoder2D.fit_transform(z_test[:1000])

plt.figure(figsize=(16, 6))
plt.subplot(121)
sns.scatterplot(*w_train.T, hue=y_train[:1000]).set_title('Train dataset')
plt.subplot(122)
sns.scatterplot(*w_test.T, hue=y_test[:1000]).set_title('Test dataset');

Nota: como esperado, as amostras selecionadas para compor o conjunto de teste parecem estar suficientemente espalhadas,
para o propósito do nosso pequeno exercício. Além disso, as proporções de rótulos presentes em treinamento e teste não estão muito distantes.

## Modelando um detector de pagamentos

**Execício (2 pts):** instancie uma máquina de vetor de suporte binário **linear** e a treine sobre o conjunto de dados de treino preprocessado `(z_train, y_train)`.

Leia mais sobre as máquinas de vetor de suporte implementadas no sklearn na página de documentação do módulo [scikit-learn.org/svm](https://scikit-learn.org/stable/modules/svm.html).

In [ ]:
# from sklearn... import ...
# model = ...
# model.fit(...)

### Avaliação do modelo treinado

Para verificar a capacidade de generalização do modelo,
devemos avaliar seu desempenho sobre o conjunto de teste previamente separado:

In [ ]:
predictions = svm.predict(z_test)

**Exercício (1 pts):** descreva o número total de acertos, a acurácia e a matriz de confusão.  
Não utilize loops! Somente operações vetoriais ou utilitários no sklearn (dica: [sklearn/metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)).

In [ ]:
# hits = ...
# acc = ...
# cm = ...

Um destes utilitários é muito útil: o `classification_report`.  
Ele condensa algumas métricas populares em um único relatório:

In [ ]:
labels = svm.predict(z_test)
report = metrics.classification_report(y_test, labels)

print(report)

**Perguntas (1 pts):**

- O modelo prediz todas as classes com alta acurácia?
- O modelo se comporta de forma similar em teste?

R:

## Modelando um detector de pagamentos no período correto e atrasados

**Exercício (1 pt):** altere o conjunto para unir as amostras `Paid` e `Late` em uma único rótulo `Paid`. O conjunto deve ser binarizado sob os rótulos `Paid` e `Not paid`. Re-treine um classificador baseado em máquina de vetor de suporte sobre esse conjunto.

In [ ]:
x = debt[features]
y = debt[target].copy()

# Modifique os rótulos em `y`...

In [ ]:
# Separa os conjuntos de treino/teste.
test_size = .5
train_samples = ceil((1 - test_size) * len(debt))
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=test_size,
                                                    random_state=8173)

# Trata valores faltantes e normaliza os dados originais.
z_train = encoder.fit_transform(x_train)
z_test = encoder.transform(x_test)

In [ ]:
# svm = ...
# svm.fit(...)

**Exercício (1 pts):** avalie o seu SVM sobre o conjunto de teste.

In [ ]:
# test_predictions = ...
# test_accuracy = ...

### Confiança nas predições

As máquinas de vetor de suporte trabalham com o conceito de distância ao hiperplano,
em vez da probabilidade usual gerada pela função logística. Nessa configuração, amostras próximas ao hiperplano apresentam uma maior similaridade com as classes vizinhas do que amostras distantes a ele.

Podemos exibir a distribuição de distâncias em treino e teste da seguinte forma:

In [ ]:
plt.figure(figsize=(12, 4))

for index, (tag, z) in enumerate((('train', z_train),
                                  ('test', z_test))):
    plt.subplot(1,2, index + 1)
    plt.title('Distribuição de confiança sobre o conjunto de %s' % tag)

    distance = svm.decision_function(np.clip(z, -2, 2))
    sns.distplot(distance, bins=20);

**Pergunta (1 pts):** considerando a distribuição de confiança sobre treino e teste, o modelo apresenta maior dificuldade para testar amostras no conjunto de teste do que no de treino?

R:


## Comparando múltiplos estimadores sobre um conjunto de dados

A fim de manter uma melhor representatividade de todo o conjunto, podemos empregar a técnica de validação cruzada para testar dois ou mais classificadores e verificar qual apresenta melhor comportamento sobre o conjunto de dados.

**Atividade (2 pts):** instancie dois ou mais classificadores binários e utilize o procedimento de validação-cruzada sobre os dados de treino. Registre o resultado do processo para cada estimador.

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
# from sklearn... import ...Classifier

cv = StratifiedKFold(n_splits=3, random_state=7129)

# estimators = [e1, e2, ...]
# results = [... for e in estimators]

**Atividade (1 pt):** selecione o estimador que apresenta maior pontuação média sobre as *folds* de validação. Treine-o sobre todo o conjunto de treino e avalie sua performance sobre o conjunto de teste.

Importante: como tomamos uma decisão considerando os dados usados na validação cruzada (qual estimador utilizar), **não devemos unir** os dados de teste com os de treinamento antes do processo.

In [ ]:
# best_ix = ...
# e = estimators[best_ix]
# treine `e`...
# avalie `e`...

**Pergunta (1 pt):** este estimador apresenta maior acurácia, comparada à máquina de vetor de suporte treinada anteriormente?

R: 